In [1]:
import numpy as np
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # Select the GPU index
import scipy.io
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import math
from PIL import Image
from collections import OrderedDict
import torchvision.transforms as T
import matplotlib.pyplot as plt
import time
from torch.optim.lr_scheduler import _LRScheduler
import warnings
import spacy
from scipy.io import savemat
from scipy import stats
import dill as pickle
import thop
from torch_challenge_dataset import DeepVerseChallengeLoaderTaskTwo
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
#Parameters
onoffdict={'GPS': False, 'CAMERAS': False, 'RADAR': False}
reduction = 4
batch_size = 200
weight_path=f'models/CSINetplustask2/cr{reduction}/gps{onoffdict["GPS"]}_cam{onoffdict["CAMERAS"]}_rad{onoffdict["RADAR"]}/'

In [3]:
reduction

4

In [4]:
weight_path

'models/CSINetplustask2/cr4/gpsFalse_camFalse_radFalse/'

In [5]:
if not os.path.exists(weight_path):
    os.makedirs(weight_path)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
train_dataset = DeepVerseChallengeLoaderTaskTwo(csv_path = r'./dataset_train.csv')
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, num_workers=5)
test_dataset = DeepVerseChallengeLoaderTaskTwo(csv_path = r'./dataset_validation.csv')
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=True, num_workers=5)

# Utils and Models

In [8]:
def CSI_abs_reshape(y, csi_std=2.8117975e-06, target_std=1.0):
    y = torch.abs(y)
    y=(y/csi_std)*target_std
    return y

In [9]:
#preprocessing output as neural network doesnot understand complex numbers, without normalization
def CSI_complex2real(y):
    ry = torch.real(y)
    iy= torch.imag(y)
    oy=torch.cat([ry,iy],dim=1)
    return oy

In [10]:
def cal_model_parameters(model):
    total_param  = []
    for p1 in model.parameters():
        total_param.append(int(p1.numel()))
    return sum(total_param)

In [11]:
class ConvLayer(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size,stride=1, activation="LeakyReLu"):
        padding = (kernel_size - 1) // 2
        dict_activation ={"LeakyReLu":nn.LeakyReLU(negative_slope=0.3,inplace=True),"Sigmoid":nn.Sigmoid(),"Tanh":nn.Tanh()}
        activation_layer = dict_activation[activation]
        super(ConvLayer, self).__init__(OrderedDict([
            ("conv", nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding=padding, bias=False)),
            ("bn", nn.BatchNorm2d(out_planes)),
            ("activation",activation_layer)
        ]))

class RefineNetBlock(nn.Module):
    def __init__(self):
        super(RefineNetBlock, self).__init__()
        self.direct = nn.Sequential(OrderedDict([
            ("conv_7x7", ConvLayer(2, 8, 7, activation="LeakyReLu")),
            ("conv_5x5", ConvLayer(8, 16, 5, activation="LeakyReLu")),
            ("conv_3x3",ConvLayer(16,2,3,activation="Tanh"))
        ]))
        self.identity = nn.Identity()
        self.relu = nn.ReLU()
    def forward(self, x):
        identity = self.identity(x)
        out = self.direct(x)
        out = self.relu(out + identity)
        
        return out

In [12]:
class task2Encoder(nn.Module):
    
    def __init__(self, reduction=16):
        super(task2Encoder, self).__init__()
        total_size, in_channel, w, h = 8192, 2, 64, 64
        self.encoder_conv = nn.Sequential(OrderedDict([
            ("conv1_7x7", ConvLayer(1, 2, 7, activation='LeakyReLu')),
            ("conv2_7x7",ConvLayer(2,2,7,activation='LeakyReLu'))
        ]))
        self.encoder_fc = nn.Linear(total_size, total_size // reduction)
         
    
    def forward(self, x):
        n,c,h,w = x.detach().size()
        out = self.encoder_conv(x.to(torch.float32))
        out = self.encoder_fc(out.view(n, -1))
        return out
       

In [13]:
class task2Decoder(nn.Module):
    
    def __init__(self, reduction=16):
        super(task2Decoder, self).__init__()
        total_size, in_channel, w, h = 8192, 2, 64, 64
        self.decoder_fc = nn.Linear(total_size // reduction, total_size)
        self.decoder_conv = ConvLayer(2, 2, 7, activation="Sigmoid")
        self.decoder_refine = nn.Sequential(OrderedDict([
            (f"RefineNet{i+1}",RefineNetBlock()) for i in range(5)
        ]))
        self.decoder_sigmoid = nn.Sigmoid()
        self.decoder_fc2 = nn.Linear(total_size, total_size//2)
        self.sig2 = nn.Sigmoid()
        
    def forward(self, Hencoded):
        bs = Hencoded.size(0)
        out = self.decoder_fc(Hencoded).view(bs, -1, 64, 64)
        out = self.decoder_conv(out)
        out = self.decoder_refine(out)
        out = self.sig2(self.decoder_fc2(out.view(bs, -1)))
        
        return out.view(bs, -1, 64, 64)

In [14]:
#complete task 2 model including encoder, decoder and channel
class task2model(nn.Module):
    def __init__(self, reduction=16):
        super().__init__()
        
        self.en=task2Encoder(reduction)
        
        self.de=task2Decoder(reduction)
        
    
   
    def forward(self, Hin, device, is_training): 
        
        #Encoder
        Hencoded=self.en(Hin)
        
        
        #Decoder   
        Hdecoded=self.de(Hencoded)
        

        return Hdecoded

In [15]:
#Loss

#criterion=nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
criterion= nn.MSELoss().to(device)

# Inference

In [16]:
def run_test(model, test_loader, device, criterion):
    num_test_batches = len(test_loader)
    model.eval()
    with torch.no_grad():
        mse1 = 0
        for b, (X_test, y_test) in enumerate(test_loader):
            y_test = y_test.to(device)
            Xin = CSI_abs_reshape(X_test[0])
            y_pred = model(Xin.to(device), device, is_training=False)
            y_test_reshaped = CSI_abs_reshape(y_test)
            mse0 = criterion(y_pred, y_test_reshaped) 
            mse1 += mse0 
        
    avg_mse = mse1 / num_test_batches
    return avg_mse.item()

In [17]:
def calculate_confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = np.mean(data)
    se = stats.sem(data)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return mean, h

In [18]:
test_dataset = DeepVerseChallengeLoaderTaskTwo(csv_path = r'./dataset_validation.csv')
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=5)

In [19]:

h_list = torch.tensor([])
for b, (x,h) in enumerate(test_loader):
    h = CSI_abs_reshape(h)
    h_list = torch.cat([h_list,h])
target_loss = torch.mean((torch.abs(h_list) - torch.mean(torch.abs(h_list))) ** 2)

In [20]:
num_runs =10

In [21]:
avg_mse_list = []
improvement_list = []
for _ in range(num_runs):
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=5)
    model = torch.load(weight_path + "task2.pth", map_location=device)
    avg_mse = run_test(model, test_loader, device, criterion)
    avg_mse_list.append(avg_mse)
    improvement = (target_loss.item() - avg_mse) / target_loss.item() * 100
    improvement_list.append(improvement)
mean_mse, margin_of_error = calculate_confidence_interval(avg_mse_list)
improvement_mean, improvement_margin_of_error = calculate_confidence_interval(improvement_list)
print(f'Percentage Improvement Mean Achieved: {improvement_mean:.4f}%')
print(f'Percentage Improvement Confidence Interval Achieved: {improvement_margin_of_error:.4f}%')
print(f"Mean MSE: {mean_mse:.4f}")
print(f"95% Confidence Interval: ({mean_mse - margin_of_error:.4f}, {mean_mse + margin_of_error:.4f})")
print(f"Margin of Error: {margin_of_error:.4f}")

Percentage Improvement Mean Achieved: 20.3023%
Percentage Improvement Confidence Interval Achieved: 1.3708%
Mean MSE: 0.8893
95% Confidence Interval: (0.8740, 0.9045)
Margin of Error: 0.0153
